In [2]:
from chinese_checkers.game import ChineseCheckersGame
from chinese_checkers.simulation import S3SimulationCatalog, GameSimulation, SimulationMetadata
from chinese_checkers.geometry import Printer

from dataclasses import asdict


from typing import List

import pandas as pd
from tqdm import tqdm
from collections import Counter

In [3]:
# filter to only 2 player games on a size 4 board
catalog = S3SimulationCatalog()
metadata_list: List[SimulationMetadata] = [
    m
    for m in catalog.list_datasets()
    if m.player_count == 2 and m.board_size == 4 and m.winning_player in ["0", "3"]
]
pd.DataFrame([asdict(m) for m in metadata_list])

,player_count,board_size,max_game_length,winning_player,name,version
0,2,4,1000,0,bad-player-3-bootstrap-simulation,v0.0.1
1,2,4,1000,3,bad-player-3-bootstrap-simulation,v0.0.1
2,2,4,1000,0,bootstrap-p3-010-simulation,v0.0.1
3,2,4,1000,3,bootstrap-p3-010-simulation,v0.0.1
4,2,4,1000,0,bootstrap-simulation-p0d05-p0d25,v0.0.1
5,2,4,1000,3,bootstrap-simulation-p0d05-p0d25,v0.0.1
6,2,4,1000,0,bootstrap-simulation-p0d15,v0.0.1
7,2,4,1000,3,bootstrap-simulation-p0d15,v0.0.1
8,2,4,1000,0,bootstrap-simulation,v0.0.1
9,2,4,1000,3,bootstrap-simulation,v0.0.1


In [4]:
simulations: List[GameSimulation] = []

with tqdm(metadata_list, desc="Loading Simulations") as progress_bar:
    for metadata in progress_bar:
        simulations += list(catalog.load_random_batch(metadata, 20))
        progress_bar.set_postfix({"Simulation Count": len(simulations)})

print(f"All {len(simulations)} simulations loaded.")

Loading Simulations: 100%|██████████| 22/22 [00:41<00:00,  1.87s/it, Simulation Count=9526]

All 9526 simulations loaded.


In [4]:
simulations[0]

GameSimulation(metadata=SimulationMetadata(player_count=2, board_size=4, max_game_length=1000, winning_player='0', name='bad-player-3-bootstrap-simulation', version='v0.0.1'), data=SimulationData(player_ids=['0', '3'], player_start_positions=[[(-3, -2), (-4, -4), (-3, -4), (-1, -4), (-2, -4), (-4, -3), (-4, -1), (-4, -2), (-3, -3), (-2, -3)], [(3, 4), (4, 2), (4, 1), (3, 2), (4, 3), (1, 4), (4, 4), (0, -1), (-2, -1), (2, 3)]], player_target_positions=[[(-1, -4), (-2, -3), (-2, -4), (-3, -2), (-3, -3), (-3, -4), (-4, -1), (-4, -2), (-4, -3), (-4, -4)], [(1, 4), (2, 3), (2, 4), (3, 2), (3, 3), (3, 4), (4, 1), (4, 2), (4, 3), (4, 4)]], historical_moves=[Move(0, -1), Position(2, 3), Move(2, 0), Position(-4, -2), Move(-2, 0), Position(3, 2), Move(0, 2), Position(-2, -3), Move(-2, -2), Position(4, 3), Move(2, 2), Position(-3, -3), Move(1, -1), Position(2, 1), Move(2, 2), Position(-3, -2), Move(-1, 0), Position(3, 0), Move(2, 0), Position(-2, -1), Move(-2, -2), Position(4, 1), Move(2, 4), Pos

In [ ]:
# compute the summary statistics for the simulations
data = []
for sim in simulations:
    metadata: SimulationMetadata = sim.metadata
    data.append({
        "name": metadata.name,
        "version": metadata.version,
        "winning_player": metadata.winning_player,
        "game_length": len(sim.data.historical_moves)
    })

simulations_df = pd.DataFrame(data)

summary_df = simulations_df.groupby(["name", "version", "winning_player"]).agg(
    game_count=("winning_player", "size"),
    avg_game_length=("game_length", "mean"),
    min_game_length=("game_length", "min"),
    max_game_length=("game_length", "max"),
).reset_index()

metadata_summary_df = simulations_df.groupby(["name", "version"]).agg(
    total_games=("winning_player", "size"),
    winning_player_counts=("winning_player", lambda x: Counter(x))
).reset_index()

full_summary_df = pd.merge(summary_df, metadata_summary_df, on=["name", "version"])
full_summary_df

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

all_moves = []
for sim in simulations:
    # if sim.metadata.name in ['bad-player-3-bootstrap-simulation']:
    all_moves.extend(sim.data.historical_moves)

move_counts = Counter(all_moves)

move_df = pd.DataFrame(list(move_counts.items()), columns=["move", "count"])

total_moves = len(all_moves)
unique_moves = move_df[move_df["count"] == 1].shape[0]
duplicate_moves = move_df[move_df["count"] > 1].shape[0]
max_duplicate_count = move_df["count"].max()
mean_duplicate_count = move_df[move_df["count"] > 1]["count"].mean()

summary_stats = {
    "total_moves": total_moves,
    "unique_moves": unique_moves,
    "duplicate_moves": duplicate_moves,
    "max_duplicate_count": max_duplicate_count,
    "mean_duplicate_count": mean_duplicate_count,
}
summary_df = pd.DataFrame([summary_stats])

print(summary_df)

plt.figure(figsize=(10, 6))
move_df[move_df["count"] > 1]["count"].plot(kind="hist", bins=200, edgecolor="black")
plt.title("Distribution of Duplicate Move Counts")
plt.xlabel("Duplicate Count")
plt.ylabel("Frequency")
plt.show()


In [ ]:
move_df.sort_values(by='count', ascending=False)

In [ ]:
move_df.sort_values(by='count', ascending=True)

In [ ]:
game = ChineseCheckersGame.start_game(2, 4)
game = ChineseCheckersGame(game.players, game.turn, game.board, Printer(show_coordinates=True))
game.print()